In [ ]:
import csv
import ast

class Player:
    def __init__(self, nick):
        self.nick = nick
        self.elo = 1000
        self.matches_played = 0
        self.wins = 0
        self.losses = 0

    def win_rate(self):
        if self.matches_played == 0:
            return 0
        return self.wins / self.matches_played


players_temp = {}
PLAYERS = []

with open("matches.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)

    players_unique = set()

    for row in reader:
        players_a = row["a"]
        players_a = ast.literal_eval(players_a)

        players_b = row["b"]
        players_b = ast.literal_eval(players_b)

        for player in players_a:
            players_unique.add(player)

        for player in players_b:
            players_unique.add(player)
        
for player in players_unique:
    players_temp[player] = Player(player)


for player in players_temp:
    PLAYERS.append(players_temp[player])


print(f"{len(PLAYERS)} graczy")

In [ ]:
class MatchBasic:
    def __init__(self, match_id, winners, losers, date):
        self.match_id = match_id
        self.winners = winners  # lista playerow
        self.losers = losers    # lista playerow
        self.date = date

class MatchPlayer:
    def __init__(self, match_id, player, elo_change, is_winner):
        self.match_id = match_id
        self.player = player    # ref do Player
        self.elo_change = elo_change
        self.elo_before = 0
        self.elo_after = 0
        self.is_winner = is_winner

class Match(MatchBasic):
    def __init__(self, match_id, winners, losers, date, match_players=None):
        super().__init__(match_id, winners, losers, date)
        self.match_players = match_players or [] 

def get_player_by_nick(nick) -> Player:
    for player in PLAYERS:
        if player.nick == nick:
            return player
        
    print(f"ERROR - Nie znaleziono gracza o nicku {nick}")
    print(len(PLAYERS))
    return None

with open("matches.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    matches = []

    for row in reader:
        match_id = row["id"]
        winner_team = row["winner"]
        date = row["date"]
        
        def safe_parse_list(s):
            try:
                return ast.literal_eval(s)
            except (ValueError, SyntaxError):
                import re
                matches = re.findall(r"'([^']*)'", s)
                if matches:
                    return matches
                return [s.strip("[]")]
        
        team_a_nicks = safe_parse_list(row["a"])
        team_b_nicks = safe_parse_list(row["b"])
        
        team_a_players = [get_player_by_nick(nick) for nick in team_a_nicks]
        team_b_players = [get_player_by_nick(nick) for nick in team_b_nicks]

        if winner_team == "a":
            winners = team_a_players
            losers = team_b_players
        else:
            winners = team_b_players
            losers = team_a_players
        
        match_players = []

        for player in winners:
            player.matches_played += 1
            player.wins += 1
            match_players.append(MatchPlayer(match_id, player, 0, True))
            
        for player in losers:
            player.matches_played += 1
            player.losses += 1
            match_players.append(MatchPlayer(match_id, player, 0, False))
        
        match = Match(match_id, winners, losers, date, match_players)
        matches.append(match)

for match in matches[:3]:
    winners_nicks = [player.nick for player in match.winners]
    losers_nicks = [player.nick for player in match.losers]
    print(f"{match.match_id}: {winners_nicks} vs {losers_nicks}")
    
    print("  Gracze w meczu:")
    for mp in match.match_players:
        player = mp.player
        status = "Wygrał" if mp.is_winner else "Przegrał"
        print(f"    {player.nick}:\t{status} (W: {player.wins}, L: {player.losses}, M: {player.matches_played})")
    print()

print(f"{len(matches)} meczy")
print(f"{len(PLAYERS)} graczy")

In [ ]:
def calculate_glicko2